In [1]:
import pandas as pd
import seaborn as sns
import networkx as nx
import numpy as np
from funding import core

In [2]:
all2all_path = "../../data/NationalFunding/Data/DerivedData/Derived/dependence/pub_noforeign_fund_all2all.csv"
cntry_region_path = '../../data/NationalFunding/Data/AdditionalData/cntry_region.xlsx'
fund_frac_path="../../data/NationalFunding/Data/DerivedData/Derived/cntry_fund_frac.csv"
backbone_net_path = "backbone.gexf"

In [3]:
# Parameters
all2all_path = "../data/NationalFunding/Data/DerivedData/Derived/dependence/pub_noforeign_fund_all2all.csv"
cntry_region_path = "../data/NationalFunding/Data/AdditionalData/cntry_region.xlsx"
fund_frac_path = "../data/NationalFunding/Data/DerivedData/Derived/cntry_fund_frac.csv"
backbone_net_path = "../data/NationalFunding/Data/DerivedData/Derived/dependence/all2all_backbone_alpha0005.gexf"


In [4]:
all2all_df = pd.read_csv(all2all_path)

In [5]:
all2all_df=all2all_df[(all2all_df.source!=all2all_df.target) & (all2all_df.p!=0)] #remove selfloop and remove 0 weight
all2all_df=all2all_df[['source','target','p']]
all2all_df.columns=['source','target','weight']

In [6]:
fund_df=pd.read_csv(fund_frac_path)
fund_df = fund_df.groupby(['funder'])['cnt'].sum().reset_index(name='count')
fund_df['cnt_log']=np.log(fund_df['count'])
fund_df.head()

,funder,count,cnt_log
0,Afghanistan,3.089135,1.127891
1,Albania,17.032612,2.835130
2,Algeria,1824.735809,7.509190
3,Andorra,27.341270,3.308397
4,Angola,24.671520,3.205650


In [7]:
cntry_region = pd.read_excel(cntry_region_path)
cntry_region = cntry_region.replace(to_replace={'region2':{'EU':'Europe','Other Europe':'Europe'}})
cntry_region.head()

,cntry,region1,eulabel,region,region2
0,Algeria,Africa,NaN,Africa,Africa
1,Angola,Africa,NaN,Africa,Africa
2,Belgian Congo,Africa,NaN,Africa,Africa
3,Benin,Africa,NaN,Africa,Africa
4,Botswana,Africa,NaN,Africa,Africa


## create network

In [8]:
net = nx.DiGraph()
for i, row in all2all_df.iterrows():
    net.add_edge(row['source'], row['target'], weight=row['weight'])

## set the attributes of nodes

In [9]:
fund_dict=dict(zip(fund_df['funder'],fund_df['cnt_log']))
region_dict=dict(zip(cntry_region['cntry'],cntry_region['region2']))

In [10]:
nx.set_node_attributes(net,0,'fund')
nx.set_node_attributes(net,0,'region')
nx.set_node_attributes(net,fund_dict,'fund')
nx.set_node_attributes(net,region_dict,'region')

In [11]:
net_alpha = core.disparity_filter(net)

In [12]:
alpha = 0.005
selected_edges = [(u, v) for u, v, attrs in net_alpha.edges(data=True) if attrs.get('alpha_in', 0) < alpha]
selected_graph = net.edge_subgraph(selected_edges)

In [13]:
nx.write_gexf(selected_graph, backbone_net_path)